In [1]:
import tensorflow as tf
import numpy as np
import argparse
import facenet
import os
import sys
import math
import pickle

In [2]:
data_dir = 'Dataset/FaceData/processed'
batch_size=1000
image_size=160
emb_array =np.array
labels =[]
with tf.Graph().as_default():
      
        with tf.compat.v1.Session() as sess:
            np.random.seed(seed=666)
            dataset = facenet.get_dataset(data_dir)
            paths, labels = facenet.get_image_paths_and_labels(dataset)
            print('Loading feature extraction model')
            facenet.load_model('Models/20180402-114759.pb')
            images_placeholder = tf.compat.v1.get_default_graph().get_tensor_by_name("input:0")
            embeddings = tf.compat.v1.get_default_graph().get_tensor_by_name("embeddings:0")
            phase_train_placeholder = tf.compat.v1.get_default_graph().get_tensor_by_name("phase_train:0")
            embedding_size = embeddings.get_shape()[1]
            # Run forward pass to calculate embeddings
            print('Calculating features for images')
            nrof_images = len(paths)
            nrof_batches_per_epoch = int(math.ceil(1.0*nrof_images / batch_size))
            emb_array = np.zeros((nrof_images, embedding_size))
            for i in range(nrof_batches_per_epoch):
                start_index = i*batch_size
                end_index = min((i+1)*batch_size, nrof_images)
                paths_batch = paths[start_index:end_index]
                images = facenet.load_data(paths_batch, False, False, image_size)
                feed_dict = { images_placeholder:images, phase_train_placeholder:False }
                emb_array[start_index:end_index,:] = sess.run(embeddings, feed_dict=feed_dict)



Loading feature extraction model
Model filename: F:\datamining_project\Models\20180402-114759.pb
Instructions for updating:
Use tf.gfile.GFile.
Calculating features for images


In [3]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(emb_array,labels,test_size=0.4,random_state=42)

In [4]:
from knn import KNN
knn_model = KNN()
knn_model.fit(x_train, y_train)

In [5]:
knn_prediction = knn_model.predict(x_test)

In [6]:
y_train_svm = np.array(y_train)
y_test_svm =np.array(y_test)
from svm import SVM
svm_model= SVM()
svm_model.train(x_train,y_train_svm)

In [7]:
svm_prediction = svm_model.predict(x_test)

In [8]:
from sklearn.metrics import accuracy_score, f1_score
accuracy_knn = accuracy_score(knn_prediction, y_test)
print("knn:",accuracy_knn)
accuracy_svm = accuracy_score(svm_prediction, y_test_svm)
print("svm:", accuracy_svm)

knn: 0.9565217391304348
svm: 1.0


In [9]:
f1_knn = f1_score(knn_prediction, y_test,average='micro')
f1_svm = f1_score(svm_prediction, y_test_svm,average='micro')
print("knn:", f1_knn)
print("svm:", f1_svm)

knn: 0.9565217391304348
svm: 1.0
